# 금융감독원 금융통계정보시스템 Open API 이용방법
* 금융업종별 금융회사들의 재무상태표와 손익계산서 등 요약재무정보를 수집할 수 있습니다.  
* 의견이나 문의는 아래 이메일로 보내주시기 바랍니다. 
  - <kevin.na74@gmail.com>

## API 인증키 신청하기
* 금융감독원 금융통계정보시스템(http://fisis.fss.or.kr) 접속
* 상단 가로 메뉴 중 "OPEN API" 클릭
* 왼쪽 세로 메뉴 중 "인증키신청" 클릭 후 신청양식을 기입
  - OPEN API 이용약관 "동의함" 선택
  - 개인정보 수집약관 "동의함" 선택
  - 이용형태 "비영리" 선택
  - 신청정보란에 이메일과 사용용도를 기입 후 "인증키신청" 클릭  
* 위 과정을 거치면 자신의 이메일로 API 인증키를 받을 수 있습니다. 

## API 구조
* 금감원 금융통계정보시스템은 상당히 복잡한 구조를 가지고 있습니다. 
* "OPEN API > API 상세"로 이동하면 4개의 하위 메뉴로 구성되어 있습니다. 
  - 통계정보 API : 우리가 최종적으로 얻고자 하는 금융통계를 수집할 수 있습니다.  
    * 요청변수로 금융회사코드, 통계코드, 계정항목코드 등을 입력해야 합니다. 
    * 이 외에 시기구분, 검색시작년월, 검색종료년월 등을 추가합니다. 
  - 금융회사 API : 금융권역별 금융회사코드, 금융회사명을 얻습니다. 
  - 통계목록 API : 금융권역별 통계코드, 통계명을 얻습니다. 
  - 계정항목 API : 통계코드별 계정항목코드, 계정항목명을 얻습니다.  
* 정리해보면, 
  - 통계정보 API로 원하는 금융통계를 수집하기 위해서 금융회사코드,  
    통계코드, 계정항목코드를 미리 받아서 정리해야 합니다. 
  - 왜 이렇게 만들어 놨을까요?? ^^

In [1]:
# 필요한 라이브러리 불러오기
library(httr)
library(rvest)
library(readr)
library(stringr)

# 공통항목 설정
main <- "http://fisis.fss.or.kr/openapi/"
auth <- "메일로 받은 인증키를 여기에 추가하세요"
lang <- "kr"

# 금융권역 분류 (금융회사 API 페이지에서 확인!)
# A:국내은행, C:신용카드사, H:생명보험, I:손해보험, K:리스사, 
# T:할부금융사, N:신기술금융사, E:상호저축은행, L:금융지주회사 등
sectorCd <- c("A","C","H","I","K","T","N","E","L")

# 금융권역별 통계 분류 (통계목록 API 페이지에서 확인)
statTbCd <- c("A","B","C","D","P")

Loading required package: xml2

Attaching package: ‘readr’

The following object is masked from ‘package:rvest’:

    guess_encoding



In [2]:
auth <- "088e6e0e9dcede7129d47d354371ffd8"

In [3]:
# 국내은행에 대해서 금융회사코드와 금융회사명을 수집합니다. 
sub1 <- "companySearch.xml"
sector <- "A"

URL <- paste(main, 
             sub1, 
             "?lang=", lang, 
             "&auth=", auth, 
             "&partDiv=", sector, 
             sep="")

# 조립한 url로 요청합니다. 
resp <- GET(URL)
resp$status_code

[1] 200

In [4]:
# resp 객체의 content를 텍스트로 추출하여 구조를 파악합니다. 
xmlObj <- content(resp, as="text", encoding="EUC-KR")
cat(xmlObj)

# 복잡하게 출력이 되었지만 xml 구조로 이루어져 있음을 알 수 있습니다. 
# row가 반복됩니다. 

<?xml version="1.0" encoding="euc-kr"?> 
<result>  
<err_cd>000</err_cd>  
<err_msg>정상</err_msg>  
<total_count>31</total_count>  
<list>  
<row>  
<finance_cd>0010927</finance_cd> 
<finance_nm>국민은행</finance_nm> 
<finance_path>국내은행/일반은행/시중은행/국민은행</finance_path> 
</row>  
<row>  
<finance_cd>0010014</finance_cd> 
<finance_nm>국민은행(구)[폐]</finance_nm> 
<finance_path>국내은행/일반은행/시중은행/국민은행(구)[폐]</finance_path> 
</row>  
<row>  
<finance_cd>0010004</finance_cd> 
<finance_nm>서울은행[폐]</finance_nm> 
<finance_path>국내은행/일반은행/시중은행/서울은행[폐]</finance_path> 
</row>  
<row>  
<finance_cd>0011625</finance_cd> 
<finance_nm>신한은행</finance_nm> 
<finance_path>국내은행/일반은행/시중은행/신한은행</finance_path> 
</row>  
<row>  
<finance_cd>0010005</finance_cd> 
<finance_nm>신한은행(구)[폐]</finance_nm> 
<finance_path>국내은행/일반은행/시중은행/신한은행(구)[폐]</finance_path> 
</row>  
<row>  
<finance_cd>0010001</finance_cd> 
<finance_nm>우리은행</finance_nm> 
<finance_path>국내은행/일반은행/시중은행/우리은행</finance_path> 
</row>  
<row>  
<finance_cd>0010003</finance_c

In [5]:
# resp 객체를 compList에 할당합니다.  
compInfo <- read_xml(resp) %>% xml_nodes("row")
compInfo

# 총 31개의 금융회사 목록을 받았습니다.

{xml_nodeset (31)}
 [1] <row>\n  <finance_cd>0010927</finance_cd>\n  <finance_nm>국민은행</finance_n ...
 [2] <row>\n  <finance_cd>0010014</finance_cd>\n  <finance_nm>국민은행(구)[폐]</fin ...
 [3] <row>\n  <finance_cd>0010004</finance_cd>\n  <finance_nm>서울은행[폐]</financ ...
 [4] <row>\n  <finance_cd>0011625</finance_cd>\n  <finance_nm>신한은행</finance_n ...
 [5] <row>\n  <finance_cd>0010005</finance_cd>\n  <finance_nm>신한은행(구)[폐]</fin ...
 [6] <row>\n  <finance_cd>0010001</finance_cd>\n  <finance_nm>우리은행</finance_n ...
 [7] <row>\n  <finance_cd>0010003</finance_cd>\n  <finance_nm>조흥은행[폐]</financ ...
 [8] <row>\n  <finance_cd>0010012</finance_cd>\n  <finance_nm>평화은행[폐]</financ ...
 [9] <row>\n  <finance_cd>0013909</finance_cd>\n  <finance_nm>하나은행</finance_n ...
[10] <row>\n  <finance_cd>0010010</finance_cd>\n  <finance_nm>하나은행(구)[폐]</fin ...
[11] <row>\n  <finance_cd>0011228</finance_cd>\n  <finance_nm>하나은행[폐]</financ ...
[12] <row>\n  <finance_cd>0010002</finance_cd>\n  <finance_nm>한국스탠다드차타드은행</fi .

In [6]:
# 금융회사이름을 벡터로 정리합니다. 
compInfo %>% xml_nodes("finance_nm") %>% xml_text()

[1] "국민은행"                 "국민은행(구)[폐]"        
 [3] "서울은행[폐]"             "신한은행"                
 [5] "신한은행(구)[폐]"         "우리은행"                
 [7] "조흥은행[폐]"             "평화은행[폐]"            
 [9] "하나은행"                 "하나은행(구)[폐]"        
[11] "하나은행[폐]"             "한국스탠다드차타드은행"  
[13] "한국씨티은행"             "한국외환은행[폐]"        
[15] "한국주택은행[폐]"         "강원은행[폐]"            
[17] "경남은행"                 "광주은행"                
[19] "대구은행"                 "부산은행"                
[21] "전북은행"                 "제주은행"                
[23] "충북은행[폐]"             "농업협동조합중앙회[폐]"  
[25] "농협은행주식회사"         "수산업협동조합중앙회[폐]"
[27] "수협은행"                 "중소기업은행"            
[29] "축산업협동조합[폐]"       "한국산업은행"            
[31] "한국수출입은행"

In [7]:
# 위와 같은 방법으로 텍스트를 추출해주는 나만의 함수를 하나 만듭니다. 
# xml_nodes() 대신 xml_node()를 사용한 것에 주의합니다. 
# xml_nodes()는 공란을 건너뜁니다. 이 경우 데이터프레임을 만들 때 에러가 발생! 
# 그러므로 공란을 NA로 반환하는 xml_node()를 대신 사용합니다. 
getXmlText <- function(x, var) {
    result <- x %>% xml_node(var) %>% xml_text()
    return(result)
}

In [8]:
# 나만의 함수를 테스트합니다. 
getXmlText(compInfo, "finance_nm")

[1] "국민은행"                 "국민은행(구)[폐]"        
 [3] "서울은행[폐]"             "신한은행"                
 [5] "신한은행(구)[폐]"         "우리은행"                
 [7] "조흥은행[폐]"             "평화은행[폐]"            
 [9] "하나은행"                 "하나은행(구)[폐]"        
[11] "하나은행[폐]"             "한국스탠다드차타드은행"  
[13] "한국씨티은행"             "한국외환은행[폐]"        
[15] "한국주택은행[폐]"         "강원은행[폐]"            
[17] "경남은행"                 "광주은행"                
[19] "대구은행"                 "부산은행"                
[21] "전북은행"                 "제주은행"                
[23] "충북은행[폐]"             "농업협동조합중앙회[폐]"  
[25] "농협은행주식회사"         "수산업협동조합중앙회[폐]"
[27] "수협은행"                 "중소기업은행"            
[29] "축산업협동조합[폐]"       "한국산업은행"            
[31] "한국수출입은행"

In [9]:
# 이제 데이터프레임으로 만듭니다. 
compList <- data.frame(sector = sector, 
                       compCd = getXmlText(compInfo, "finance_cd"),
                       compNm = getXmlText(compInfo, "finance_nm"),
                       compPA = getXmlText(compInfo, "finance_path"))

head(compList)

sector,compCd,compNm,compPA
A,0010927,국민은행,국내은행/일반은행/시중은행/국민은행
A,0010014,국민은행(구)[폐],국내은행/일반은행/시중은행/국민은행(구)[폐]
A,0010004,서울은행[폐],국내은행/일반은행/시중은행/서울은행[폐]
A,0011625,신한은행,국내은행/일반은행/시중은행/신한은행
A,0010005,신한은행(구)[폐],국내은행/일반은행/시중은행/신한은행(구)[폐]
A,0010001,우리은행,국내은행/일반은행/시중은행/우리은행


In [10]:
# 이제 관심있는 모든 금융권역에 대해서 금융회사 리스트를 수집합니다. 
sub1 <- "companySearch.xml"
compList <- data.frame()

for (sector in sectorCd) {
  tryCatch({
    URL <- paste(main, 
                 sub1, 
                 "?lang=", lang, 
                 "&auth=", auth, 
                 "&partDiv=", sector, 
                 sep="")
    
    resp <- GET(URL)
    compInfo <- read_xml(resp) %>% xml_nodes("row")
    df <- data.frame(sector = sector, 
                     compCd = getXmlText(compInfo, "finance_cd"),
                     compNm = getXmlText(compInfo, "finance_nm"),
                     compPa = getXmlText(compInfo, "finance_path"))
    
    compList <- rbind(compList, df)
  }, error=function(e) e)
}

str(compList)

'data.frame':	429 obs. of  4 variables:
 $ sector: chr  "A" "A" "A" "A" ...
 $ compCd: chr  "0010927" "0010014" "0010004" "0011625" ...
 $ compNm: chr  "국민은행" "국민은행(구)[폐]" "서울은행[폐]" "신한은행" ...
 $ compPa: chr  "국내은행/일반은행/시중은행/국민은행" "국내은행/일반은행/시중은행/국민은행(구)[폐]" "국내은행/일반은행/시중은행/서울은행[폐]" "국내은행/일반은행/시중은행/신한은행" ...


In [11]:
# 폐업구분 컬럼 생성하기
lastStr <- substr(compList$compNm, nchar(compList$compNm)-2, nchar(compList$compNm))
compList$status <- ifelse(lastStr=="[폐]","폐업","운영")
head(compList)

sector,compCd,compNm,compPa,status
A,0010927,국민은행,국내은행/일반은행/시중은행/국민은행,운영
A,0010014,국민은행(구)[폐],국내은행/일반은행/시중은행/국민은행(구)[폐],폐업
A,0010004,서울은행[폐],국내은행/일반은행/시중은행/서울은행[폐],폐업
A,0011625,신한은행,국내은행/일반은행/시중은행/신한은행,운영
A,0010005,신한은행(구)[폐],국내은행/일반은행/시중은행/신한은행(구)[폐],폐업
A,0010001,우리은행,국내은행/일반은행/시중은행/우리은행,운영


In [12]:
# 회사경로(finance_path) 정리 
# 금융업종마다 경로의 깊이가 다름 
path <- str_split(compList$compPa,"/")

for (i in 1:length(path)) {
  compList$depth[i] <- length(path[[i]])
}
           
# 대/중/소분류 컬럼 생성하고, 불필요한 컬럼값은 NA로 치환하기
compList$path1 <- sapply(path, "[", 1)
compList$path2 <- sapply(path, "[", 2)
compList$path3 <- sapply(path, "[", 3)

# 공백을 NA로 치환 
compList$path3[compList$depth==3] <- NA
compList$path2[compList$depth==2] <- NA 
           
head(compList)

sector,compCd,compNm,compPa,status,depth,path1,path2,path3
A,0010927,국민은행,국내은행/일반은행/시중은행/국민은행,운영,4,국내은행,일반은행,시중은행
A,0010014,국민은행(구)[폐],국내은행/일반은행/시중은행/국민은행(구)[폐],폐업,4,국내은행,일반은행,시중은행
A,0010004,서울은행[폐],국내은행/일반은행/시중은행/서울은행[폐],폐업,4,국내은행,일반은행,시중은행
A,0011625,신한은행,국내은행/일반은행/시중은행/신한은행,운영,4,국내은행,일반은행,시중은행
A,0010005,신한은행(구)[폐],국내은행/일반은행/시중은행/신한은행(구)[폐],폐업,4,국내은행,일반은행,시중은행
A,0010001,우리은행,국내은행/일반은행/시중은행/우리은행,운영,4,국내은행,일반은행,시중은행


In [13]:
# 컬럼 정리하기
compList <- compList[,-4]
head(compList)

sector,compCd,compNm,status,depth,path1,path2,path3
A,0010927,국민은행,운영,4,국내은행,일반은행,시중은행
A,0010014,국민은행(구)[폐],폐업,4,국내은행,일반은행,시중은행
A,0010004,서울은행[폐],폐업,4,국내은행,일반은행,시중은행
A,0011625,신한은행,운영,4,국내은행,일반은행,시중은행
A,0010005,신한은행(구)[폐],폐업,4,국내은행,일반은행,시중은행
A,0010001,우리은행,운영,4,국내은행,일반은행,시중은행


In [14]:
# 2. 통계목록 내려받기
# 금융권역별 통계목록이 서로 다르기 때문에 for-loop를 2번 작성합니다. 
sub2 <- "statisticsListSearch.xml"
statList <- data.frame()

for (sector in sectorCd) {
  for (statTb in statTbCd) {
    tryCatch({
      URL <- paste(main, 
                   sub2, 
                   "?lang=", lang, 
                   "&auth=", auth, 
                   "&lrgDiv=", sector, 
                   "&smlDiv=", statTb, 
                   sep="")
      
      resp <- GET(URL)
      statInfo <- read_xml(resp) %>% xml_nodes("row")
      df <- data.frame(sector = sector, 
                       sectNm = getXmlText(statInfo, "lrg_div_nm"),
                       statTb = statTb, 
                       statDv = getXmlText(statInfo, "sml_div_nm"),
                       statCd = getXmlText(statInfo, "list_no"),
                       statNm = getXmlText(statInfo, "list_nm"))
      
      statList <- rbind(statList, df)
    }, error=function(e) e)
  }
}

str(statList)
head(statList)

'data.frame':	321 obs. of  6 variables:
 $ sector: chr  "A" "A" "A" "A" ...
 $ sectNm: chr  "국내은행" "국내은행" "국내은행" "국내은행" ...
 $ statTb: chr  "A" "A" "A" "B" ...
 $ statDv: chr  "일반현황" "일반현황" "일반현황" "재무현황" ...
 $ statCd: chr  "SA001" "SA002" "SA026" "SA003" ...
 $ statNm: chr  "임직원현황" "영업점포현황" "자동화기기 설치현황" "요약재무상태표(자산-은행계정)" ...


sector,sectNm,statTb,statDv,statCd,statNm
A,국내은행,A,일반현황,SA001,임직원현황
A,국내은행,A,일반현황,SA002,영업점포현황
A,국내은행,A,일반현황,SA026,자동화기기 설치현황
A,국내은행,B,재무현황,SA003,요약재무상태표(자산-은행계정)
A,국내은행,B,재무현황,SA004,요약재무상태표(부채 및 자본-은행계정)
A,국내은행,B,재무현황,SA005,난외계정(은행계정)


In [15]:
# 3. 계정항목 내려받기 
# 다섯자리 통계코드별 하위 계정항목코드를 수집합니다. 
# 다행인 것은 같은 계정항목명이라도 금융권역별로 계정항목코드가 서로 다릅니다. 
sub3 <- "accountListSearch.xml"
accounts <- data.frame()
statCds <- statList$statCd

for (statCd in statCds) {
  tryCatch({
    URL <- paste(main, 
                 sub3, 
                 "?lang=", lang, 
                 "&auth=", auth, 
                 "&listNo=", statCd, 
                 sep="")
    
    resp <- GET(URL)
    acntInfo <- read_xml(resp) %>% xml_nodes("row")
    df <- data.frame(statCd = getXmlText(acntInfo, "list_no"),
                     statNm = getXmlText(acntInfo, "list_nm"),
                     acntCd = getXmlText(acntInfo, "account_cd"),
                     acntNm = getXmlText(acntInfo, "account_nm"))
    
    accounts <- rbind(accounts, df)
  }, error=function(e) e)
}

str(accounts)
head(accounts)

'data.frame':	3890 obs. of  4 variables:
 $ statCd: chr  "SA001" "SA001" "SA001" "SA001" ...
 $ statNm: chr  "임직원현황" "임직원현황" "임직원현황" "임직원현황" ...
 $ acntCd: chr  "A1" "A11" "A12" "A13" ...
 $ acntNm: chr  "임원" "임원_상임임원" "임원_비상임임원" "임원_이사대우" ...


statCd,statNm,acntCd,acntNm
SA001,임직원현황,A1,임원
SA001,임직원현황,A11,임원_상임임원
SA001,임직원현황,A12,임원_비상임임원
SA001,임직원현황,A13,임원_이사대우
SA001,임직원현황,A2,일반직원
SA001,임직원현황,A21,일반직원_책임자


## 최종 데이터 내려받기

In [16]:
# 금융권역별 금융회사 개수 확인
table(compList$path1)


  국내은행     리스사   생명보험   손해보험   저축은행 할부금융사 
        31         50         37         43        229         39 

In [17]:
# 국내은행에 대해 금융통계 데이터를 수집합니다. 
targetSector <- "국내은행"
term <- "Y"               # Y:연도, H:반기, Q:분기
strMonth <- "201601"      # 검색시작년월
endMonth <- "201612"      # 검색종료년월

sub4 <- "statisticsInfoSearch.xml"
result <- data.frame()

# 국내은행에 속한 금융회사코드 리스트를 설정합니다. 
# 만약 관심 있는 모든 금융권역에 대해 수집하려면 아래 명령문을 수행하는 대신,
# 가장 바깥쪽에 금융권역을 순환하는 for-loop를 하나 더 만들어야 합니다. 
comps <- compList[compList$path1==targetSector,"compCd"]

# 예시로 국내은행 중 "국민은행"에 대해서만 아래 작업을 실행시키도록 하겠습니다. 
comps <- comps[1]

for (comp in comps) {
  stats <- statList[statList$sectNm==targetSector,"statCd"]
  
  for (stat in stats) {
    acnts <- accounts[accounts$statCd==stat,"acntCd"]

    for (acnt in acnts) {
      tryCatch({
        URL <- paste(main, 
                     sub4, 
                     "?lang=", lang, 
                     "&term=", term, 
                     "&auth=", auth, 
                     "&financeCd=", comp,
                     "&listNo=", stat, 
                     "&accountCd=", acnt, 
                     "&startBaseMm=", strMonth, 
                     "&endBaseMm=", endMonth, 
                     sep="")

        resp <- GET(URL)
        finalDat <- read_xml(resp) %>% xml_nodes("row")
        
        # 금융권역과 통계코드는 API에서 제공하지 않습니다. 
        df <- data.frame(baseYm = getXmlText(finalDat, "base_month"),
                         sector = targetSector, 
                         compCd = getXmlText(finalDat, "finance_cd"),
                         compNm = getXmlText(finalDat, "finance_nm"),
                         statCd = stat,
                         acntCd = getXmlText(finalDat, "account_cd"),
                         acntNm = getXmlText(finalDat, "account_nm"),
                         statVl = getXmlText(finalDat, "a"))
    
        result <- rbind(result, df)
      }, error=function(e) e)
    }
  }
}

str(result)
head(result)

'data.frame':	669 obs. of  8 variables:
 $ baseYm: chr  "201612" "201612" "201612" "201612" ...
 $ sector: chr  "국내은행" "국내은행" "국내은행" "국내은행" ...
 $ compCd: chr  "0010927" "0010927" "0010927" "0010927" ...
 $ compNm: chr  "국민은행" "국민은행" "국민은행" "국민은행" ...
 $ statCd: chr  "SA001" "SA001" "SA001" "SA001" ...
 $ acntCd: chr  "A1" "A11" "A12" "A13" ...
 $ acntNm: chr  "임원" "임원_상임임원" "임원_비상임임원" "임원_이사대우" ...
 $ statVl: chr  "59" "2" "4" "53" ...


baseYm,sector,compCd,compNm,statCd,acntCd,acntNm,statVl
201612,국내은행,0010927,국민은행,SA001,A1,임원,59
201612,국내은행,0010927,국민은행,SA001,A11,임원_상임임원,2
201612,국내은행,0010927,국민은행,SA001,A12,임원_비상임임원,4
201612,국내은행,0010927,국민은행,SA001,A13,임원_이사대우,53
201612,국내은행,0010927,국민은행,SA001,A2,일반직원,19458
201612,국내은행,0010927,국민은행,SA001,A21,일반직원_책임자,11023


In [18]:
# 컬럼명 바꾸기
#colnames(result) <- c("기준년월","회사코드","회사이름","통계코드","계정코드","계정항목","계정값A")

In [19]:
# 통계이름 붙이고 컬럼을 정렬합니다. 
result <- merge(result, statList[,c(5,6)], by="statCd", all.x=T)
result <- result[,c(2:5,1,9,6:8)]

# 데이터 확인
head(result)

baseYm,sector,compCd,compNm,statCd,statNm,acntCd,acntNm,statVl
201612,국내은행,0010927,국민은행,SA001,임직원현황,A1,임원,59
201612,국내은행,0010927,국민은행,SA001,임직원현황,A11,임원_상임임원,2
201612,국내은행,0010927,국민은행,SA001,임직원현황,A12,임원_비상임임원,4
201612,국내은행,0010927,국민은행,SA001,임직원현황,A13,임원_이사대우,53
201612,국내은행,0010927,국민은행,SA001,임직원현황,A2,일반직원,19458
201612,국내은행,0010927,국민은행,SA001,임직원현황,A21,일반직원_책임자,11023


In [20]:
# 중복제거 전후 건수를 비교합니다.
nrow(result)
nrow(unique(result))

[1] 669

[1] 669

In [21]:
# 작업한 날짜를 지정합니다. 
today <- gsub("-","",Sys.Date())
today

# 수집한 데이터를 엑셀로 저장합니다. 
filename <- paste0("금감원_금융통계정보시스템_", paste0(endMonth,term), "_", today, ".xlsx")
xlsx::write.xlsx(result, filename, sheetName=targetSector, row.names=F)

[1] "20170920"

# End of Document